<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/af_relax_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#af_relax_design (WIP)


**Efficient and scalable de novo protein design using a relaxed sequence space**

Christopher Josef Frank, Ali Khoshouei, Yosta de Stigter, Dominik Schiewitz, Shihao Feng, Sergey Ovchinnikov, Hendrik Dietz

doi: https://doi.org/10.1101/2023.02.24.529906

**<font color="red">WARNING</font>** This notebook is in development, we are still working on adding all the options from the manuscript above.

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.0")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model

from IPython.display import HTML
from google.colab import files
import numpy as np

import requests, time
if not os.path.isfile("TMscore"):
  os.system("wget -qnc https://zhanggroup.org/TM-score/TMscore.cpp")
  os.system("g++ -static -O3 -ffast-math -lm -o TMscore TMscore.cpp")
def tmscore(x,y):
  # pass to TMscore
  output = os.popen(f'./TMscore {x} {y}')
  # parse outputs
  parse_float = lambda x: float(x.split("=")[1].split()[0])
  o = {}
  for line in output:
    line = line.rstrip()
    if line.startswith("RMSD"): o["rms"] = parse_float(line)
    if line.startswith("TM-score"): o["tms"] = parse_float(line)
    if line.startswith("GDT-TS-score"): o["gdt"] = parse_float(line)
  return o
def esmfold_api(sequence):
  esmfold_api_url = 'https://api.esmatlas.com/foldSequence/v1/pdb/'
  r = requests.post(esmfold_api_url, data=sequence)
  while r.status_code != 200:
    time.sleep(5)
    r = requests.post(esmfold_api_url, data=sequence)
  structure = r.text
  return structure

## hallucination
For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).

In [ ]:
LENGTH = 200 #@param {type:"integer"}
COPIES = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw"}
MODE = "paper" #@param ["paper", "old"]

In [ ]:
import jax
import jax.numpy as jnp
from colabdesign.af.alphafold.common import residue_constants

def rg_loss(inputs, outputs):
  positions = outputs["structure_module"]["final_atom_positions"]
  ca = positions[:,residue_constants.atom_order["CA"]]
  if MODE == "paper":
    ca = ca[::5]
  rg = jnp.sqrt(jnp.square(ca - ca.mean(0)).sum(-1).mean() + 1e-8)
  if MODE == "old":
    rg_th = 2.38 * ca.shape[0] ** 0.365
    rg = jax.nn.elu(rg - rg_th)
  return {"rg":rg}

In [ ]:
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", loss_callback=rg_loss)
mpnn_model = mk_mpnn_model()

af_model.opt["weights"]["rg"] = 0.1
af_model.prep_inputs(length=LENGTH, copies=COPIES)

print("length",af_model._lengths)

In [ ]:
af_model.restart()
if MODE == "paper":
  af_model.set_seq(mode=["gumbel","soft"])

if MODE == "old":
  # pre-design with gumbel initialization and softmax activation
  af_model.set_weights(plddt=0.0, pae=0.0, rg=0.1, con=1.0, helix=0.0)
  af_model.set_seq(mode=["gumbel"])
  af_model.design_soft(50)
  af_model.set_seq(af_model.aux["seq"]["pseudo"])

af_model.set_weights(plddt=1.0, pae=1.0, rg=0.1, con=1.0, helix=0.0)
af_model.design_logits(40)
af_model.design_logits(10, save_best=True)

In [ ]:
af_model.save_pdb(f"{af_model.protocol}.pdb")
af_model.plot_pdb()

In [ ]:
HTML(af_model.animate())

In [ ]:
af_model.get_seqs()

#Redesign with ProteinMPNN

In [ ]:
from colabdesign.shared.protein import alphabet_list as chain_list
mpnn_model.prep_inputs(pdb_filename=f"{af_model.protocol}.pdb",
                       chain=",".join(chain_list[:COPIES]),
                       homooligmer=COPIES>1,
                       rm_aa="C")
out = mpnn_model.sample(num=1, batch=8)

In [ ]:
for seq,score in zip(out["seq"],out["score"]):
  print(score,seq.split("/")[0])

#ESMfold TEST

In [ ]:
print("# rmsd tmscore sequence")
best = {}
best_rmsd = None
for n,seq in enumerate(out["seq"]):
  x = seq.split("/")[0]
  with open(f"{af_model.protocol}.{n}.pdb","w") as handle:
    pdb_str = esmfold_api(x)
    handle.write(pdb_str)
  o = tmscore(f"{af_model.protocol}.pdb",
              f"{af_model.protocol}.esmfold.{n}.pdb")
  print(n,o["rms"],o["tms"],x)
  if best_rmsd is None or o["rms"] < best_rmsd:
    best_rmsd = o["rms"] 
    best = {**o,"seq":x}

In [ ]:
best